# Scraping Yahoo Finance

## Week 7. Practice Programming Assignment 1

In this assignment you are required to look at historical data for 30 companies from [Dow Jones Index](https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average). Tickers for companies from the index can be found in *dow_jones_tickers.txt*. For each company you should get historical daily stock prices for 2019 from https://finance.yahoo.com/, and then use the data to answer the questions you will find below. 

###### Import coursera grader tools

In [1]:
import sys
sys.path.append("..")
import grading
grader = grading.Grader(assignment_key="FpZrXMbETcuStX7z6jFv2Q", 
                      all_parts=["RbP2k", "LLRai", "hgIbw", "Tx3OG", "Y05pG",
                                 "lFPeF", "htUtf", "Xri0I", "4JfUm", "oD7pP"])

In [12]:
# token expires every 30 min
COURSERA_EMAIL = 'fancycreative246@gmail.com'
COURSERA_TOKEN = 'zHXrbkM81MmNn7Kq'

<br><br><br><br>

### Coding part

In [3]:
# you can start your web-scraping here
from selenium import webdriver
import requests
import re
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

In [101]:
driver = webdriver.Chrome("/Users/joyceliu/Downloads/chromedriver")
mainSite = driver.get('https://finance.yahoo.com/')
def downloadCSV(stock):
    searchInput = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'yfin-usr-qry')))
    searchInput.send_keys("{0}".format(stock))
    time.sleep(10)
    searchInput.send_keys(Keys.ENTER)
    
    
    time.sleep(10)
    histData = driver.find_element_by_xpath('//span[text()="Historical Data"]').click()
    
    drop_down = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//span[text()="Oct 30, 2019 - Oct 30, 2020"]')))
    drop_down.click()
    
    start_date = driver.find_element(By.XPATH, '//input[@name="startDate"]')
    start_date.send_keys('01/01/2019')

    end_date = driver.find_element(By.XPATH, '//input[@name= "endDate"]')
    end_date.send_keys('01/01/2020')

    done_button = driver.find_element(By.XPATH, '//span[text()="Done"]')
    done_button.click()

    apply_button = driver.find_element(By.XPATH, '//span[text()="Apply"]')
    apply_button.click()

    download_link = driver.find_element(By.XPATH, '//span[text()="Download"]')
    download_link.click()

In [2]:
dow_jones_companies = []

with open('dow_jones_tickers.txt') as f:
    for ticker in f:
        dow_jones_companies.append(ticker.strip())

In [5]:
print(dow_jones_companies)
dow_jones_companies

['AXP', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'XOM', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'RTX', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


['AXP',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'XOM',
 'GS',
 'HD',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PFE',
 'PG',
 'TRV',
 'UNH',
 'RTX',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [104]:
for stock in dow_jones_companies:
    downloadCSV(stock)

In [4]:
import pandas as pd

In [67]:
axp_df = pd.read_csv('AXP.csv')
axp_df.head(5)
open = axp_df.iloc[0]['Open']
close = axp_df.iloc[-1]['Close']
print(open, close)

93.910004 124.489998


In [8]:
(close/open-1)*100

32.56308454634929

In [87]:
axp_df['month'] = axp_df['Date'].str.split('-', expand = True)[1].astype(int)
ret = []
# axp_df.head(5)
for i in range(1, 13):
    open = axp_df.loc[axp_df['month'] == i].iloc[0]['Open']
    close = axp_df.loc[axp_df['month'] == i].iloc[-1]['Close']
    ret.append((close/open-1)*100)
print(ret)

[9.36001770375816, 5.2045638041377185, 0.8302592097738204, 6.292503343072764, -2.349537734319085, 7.7514002749895505, -0.7422186574089773, -2.7470260017862014, -1.318206229616159, -1.1962915213889924, 1.4441338926058922, 3.474357966492536]


In [7]:
def average_change_of_price(df):
    df = pd.read_csv(stock + '.csv')
    open = df.iloc[0]['Open']
    close = df.iloc[-1]['Close']
    return (close/open-1)*100

def calculate_rows(df):
    return len(df.index)

def get_largest_volume(df):
    return df['Volume'].max()

def get_largest_daily_increase(df):
    df['daily_increase'] = (df['Close']/df['Open'] - 1)*100
    return (df['daily_increase'].max(), df['daily_increase'].min())

def get_average_monthly_price_increase(df):
    df['month'] = df['Date'].str.split('-', expand = True)[1].astype(int)
    ret = []
    for i in range(1, 13):
        open = df.loc[df['month'] == i].iloc[0]['Open']
        close = df.loc[df['month'] == i].iloc[-1]['Close']
        ret.append((close/open-1)*100)
    return ret

In [9]:
ret = []
maxValue = 0
company = ''
numRows = 0
minValue = float('inf')
company2 = ''
maxVolume = 0
company3 = ''
max_daily_increase = 0
company4 = ''
min_daily_decrease = float('inf')
company5 = ''

for stock in dow_jones_companies:
    df = pd.read_csv(stock + '.csv')
    ans = average_change_of_price(df)
    numRows += calculate_rows(df)
    volume = get_largest_volume(df)
    daily_increase, daily_decrease = get_largest_daily_increase(df)
    ret.append(ans)
    
    if maxValue < ans:
        maxValue = ans
        company = stock

    if minValue > ans:
        minValue = ans
        company2 = stock
    
    if volume > maxVolume:
        maxVolume = volume
        company3 = stock
    
    if daily_increase > max_daily_increase:
        max_daily_increase = daily_increase
        company4 = stock
    
    if daily_decrease < min_daily_decrease:
        min_daily_decrease = daily_decrease
        company5 = stock
        print(company5, daily_decrease)

print(sum(ret)/len(ret))
print(ret)
print(company)
print(company2)
print(numRows)
print(company3)
print(company4)
print(company5)

AXP -3.475408196721308
BA -6.77506775067751
UNH -7.163559664794761
23.664689872548923
[32.56308454634929, 89.58615275356708, 3.0266636957104076, 19.067962743432744, 13.434248189078723, 12.269430306295149, 3.6080194092953155, 39.919667864423204, 28.67833126658228, 19.667878409644146, 30.221930596647752, 13.845305008768216, 17.916487386375945, 45.284000373652944, 12.656060939374925, -6.069645711385796, 20.799569387706597, 58.412850072942724, 39.18120154992166, -9.137289172942697, 37.20751771072743, 16.563111185004864, 19.991841224489782, 42.41156899627585, 9.330487891737892, 44.53845692307692, -12.261902333120645, 29.681358900931464, 33.79279155953363, 3.7535545023696715]
AAPL
WBA
7507
AAPL
BA
UNH


In [90]:
ans = []
for stock in dow_jones_companies[:-1]:
    df = pd.read_csv(stock + '.csv')
    ans.append(get_average_monthly_price_increase(df))
print(ans)

[[9.36001770375816, 5.2045638041377185, 0.8302592097738204, 6.292503343072764, -2.349537734319085, 7.7514002749895505, -0.7422186574089773, -2.7470260017862014, -1.318206229616159, -1.1962915213889924, 1.4441338926058922, 3.474357966492536], [7.456907482729691, 3.70746508349471, 8.991276107413348, 4.71195992485911, -16.585665778813308, 12.710701015457815, 4.857996751488902, -2.4123366961135706, 8.496831216270163, 10.525608547541342, 7.097062021225042, 9.87017068298741], [21.958313849531528, 13.946804820393343, -14.481737080295575, -2.102121890164499, -9.753523920834617, 7.631578085218993, -6.495289321211217, 6.48708482949214, 7.444019405439795, -10.94839053869351, 7.51343178030397, -11.256396007227709], [7.361126399751083, 3.239868475079799, -2.4269018981211254, 1.2564427154267976, -14.163922995215328, 13.660235554658119, -5.477390406086357, -9.601944010953689, 7.133160184339937, 8.163265962900068, 3.8458787345152734, 1.707986261184291], [11.849579277426203, 9.2885815767064, 3.82692692

<br><br>

### Questions

<br><br>

**Question 1.** What is the average change of price over the year (in %)?

*Note 1*. The opening price is the price at which a stock first trades upon the opening of an exchange on a trading day.

*Note 2*. The closing price for any stock is the final price at which it trades during regular market hours on any given day.

*Note 3*. Here by the price change we going to mean a ratio of a closing price in the last day of the period to an opening price in the first day of that period, subtracted one and multiplied by 100.

Example: if a price of a stock in day 1 opened at \\$100, and its close price in the last day was \\$120, then the price change during the period is: $$\left(\dfrac{120}{100}-1\right) * 100 = (1.2 - 1) * 100=20.$$

The price grew by 20%.

In [14]:
answer_part_1 = 23.664689872548923

In [15]:
# Setting our answers to grader. Do not change!
grader.set_answer("RbP2k", answer_part_1)

<br>

**Question 2.** What company's stock price grew the most (in %)? Enter ticker of the company as an answer)

In [16]:
answer_part_2 = 'AAPL'

In [17]:
# Setting our answers to grader. Do not change!
grader.set_answer("LLRai", answer_part_2)

<br>

**Question 3.** What company's stock lost in price the most (in %)? Enter ticker of the company as an answer

In [18]:
answer_part_3 = 'WBA'

In [19]:
# Setting our answers to grader. Do not change!
grader.set_answer("hgIbw", answer_part_3)

<br>

**Question 4.** What company had the largest summary volume over the year? Enter ticker of the company as an answer

In [20]:
answer_part_4 = 'AAPL'

In [21]:
# Setting our answers to grader. Do not change!
grader.set_answer("Tx3OG", answer_part_4)

<br>

**Question 5.** What is the biggest stock price daily increase (in %)? Enter the number 

In [22]:
answer_part_5 = 7.741003003612024

In [23]:
# Setting our answers to grader. Do not change!
grader.set_answer("Y05pG", answer_part_5)

<br><br>

**Question 6.** What is the company that had the biggest stock price daily increase? Enter ticker of the company as an answer

In [31]:
answer_part_6 = 'BA'

In [32]:
# Setting our answers to grader. Do not change!
grader.set_answer("lFPeF", answer_part_6)

<br>

**Question 7.** What is the biggest stock price daily decrease (in %)? Enter the number

In [33]:
answer_part_7 = -7.163559664794761

In [34]:
# Setting our answers to grader. Do not change!
grader.set_answer("htUtf", answer_part_7)

<br>

**Question 8.** What is the company that had the biggest stock price daily decrese (in %)? Enter ticker of the company as an answer

In [24]:
answer_part_8 = 'UNH'

In [25]:
# Setting our answers to grader. Do not change!
grader.set_answer("Xri0I", answer_part_8)

<br>

**Question 9.** What was the best month for all companies (i.e. average monthly price increase was the best)? Enter one of the following: January, February, March, April, May, June, July, August, September, October, November, December

In [26]:
answer_part_9 = 'January'

In [27]:
# Setting our answers to grader. Do not change!
grader.set_answer("4JfUm", answer_part_9)

<br>

**Question 10.** What was the worst month for all companies (i.e. average monthly price increase was the worst)? Enter one of the following: January, February, March, April, May, June, July, August, September, October, November, December

In [28]:
answer_part_10 = 'May'

In [29]:
# Setting our answers to grader. Do not change!
grader.set_answer("oD7pP", answer_part_10)

<br>
<br>

### Submitting answers

In [35]:
# you can make submission with answers so far to check yourself at this stage
grader.submit(COURSERA_EMAIL, COURSERA_TOKEN)

Submitted to Coursera platform. See results on assignment page!
